In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import theano
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import Imputer 
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('data/train.csv')
X_test = pd.read_csv('data/test.csv')

In [4]:
# Proprocessing, building bigger classes and get rid of NaN values.

dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)
dataset["Title"].value_counts()
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

dataset_title = [i.split(",")[1].split(".")[0].strip() for i in X_test["Name"]]
X_test["Title"] = pd.Series(dataset_title)
X_test["Title"].value_counts()
X_test["Title"] = X_test["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

dataset["FamilyS"] = dataset["SibSp"] + dataset["Parch"] + 1
X_test["FamilyS"] = X_test["SibSp"] + X_test["Parch"] + 1



In [6]:
print(dataset)


     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

In [7]:
def family(x):
    if x < 2:
        return "Single"
    elif x <= 2:
        return "Couple"
    elif x <= 4:
        return "InterM"
    else:
        return "Large"
dataset["FamilyS"] = dataset["FamilyS"].apply(family)
X_test["FamilyS"] = X_test["FamilyS"].apply(family)


In [8]:
dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)
X_test['Embarked'].fillna(X_test['Embarked'].mode()[0], inplace = True)
dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
X_test['Age'].fillna(X_test['Age'].median(), inplace = True)
X_test['Fare'].fillna(X_test['Fare'].median(), inplace = True)





In [9]:
dataset = dataset.drop(["PassengerId", "Cabin","Name", "SibSp", "Parch", "Ticket"], axis=1)
X_test = X_test.drop(["PassengerId", "Cabin","Name", "SibSp", "Parch", "Ticket"], axis=1)


In [10]:
X_train = dataset.iloc[:, 1:9].values
y_train = dataset.iloc[:, 0].values
X_test = X_test.values


In [11]:
## Male and Female are still labeled, so first they have to be labelled to numbers.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X_train[:, 1] = labelencoder_X_1.fit_transform(X_train[:, 1])
X_train[:, 4] = labelencoder_X_1.fit_transform(X_train[:, 4])
X_train[:, 5] = labelencoder_X_1.fit_transform(X_train[:, 5])
X_train[:, 6] = labelencoder_X_1.fit_transform(X_train[:, 6])

labelencoder_X_2 = LabelEncoder()
X_test[:, 1] = labelencoder_X_2.fit_transform(X_test[:, 1])
X_test[:, 4] = labelencoder_X_2.fit_transform(X_test[:, 4])
X_test[:, 5] = labelencoder_X_2.fit_transform(X_test[:, 5])
X_test[:, 6] = labelencoder_X_2.fit_transform(X_test[:, 6])


In [12]:
### Since Neural Networks can not handle categorical data, i will use dummy variables
onehotencoder = OneHotEncoder(categorical_features = [0, 1, 4, 5, 6])
X_train = onehotencoder.fit_transform(X_train).toarray()

onehotencoder = OneHotEncoder(categorical_features = [0, 1, 4, 5, 6])
X_test = onehotencoder.fit_transform(X_test).toarray()


/Users/ryan/anaconda3/envs/py36-pytext/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ryan/anaconda3/envs/py36-pytext/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/Users/ryan/anaconda3/envs/py36-pytext/lib/python3.6/site-packages/s

In [23]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 19))
    classifier.add(Dropout(p = 0.2))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(p = 0.2))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 25, epochs = 1000)



In [24]:
classifier.fit(X_train, y_train)



/Users/ryan/anaconda3/envs/py36-pytext/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
  after removing the cwd from sys.path.
/Users/ryan/anaconda3/envs/py36-pytext/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.2)`
  


Epoch 1/1000
891/891 [==============================] - 0s 485us/step - loss: 0.6913 - acc: 0.6117
Epoch 2/1000
891/891 [==============================] - 0s 49us/step - loss: 0.6870 - acc: 0.6442
Epoch 3/1000
891/891 [==============================] - 0s 51us/step - loss: 0.6807 - acc: 0.6790
Epoch 4/1000
891/891 [==============================] - 0s 48us/step - loss: 0.6723 - acc: 0.6633
Epoch 5/1000
891/891 [==============================] - 0s 51us/step - loss: 0.6540 - acc: 0.6958
Epoch 6/1000
891/891 [==============================] - 0s 46us/step - loss: 0.6304 - acc: 0.6622
Epoch 7/1000
891/891 [==============================] - 0s 45us/step - loss: 0.6142 - acc: 0.6790
Epoch 8/1000
891/891 [==============================] - 0s 53us/step - loss: 0.5912 - acc: 0.6992
Epoch 9/1000
891/891 [==============================] - 0s 54us/step - loss: 0.5905 - acc: 0.7059
Epoch 10/1000
891/891 [==============================] - 0s 48us/step - loss: 0.5717 - acc: 0.7172
Epoch 11/1000
891/

891/891 [==============================] - 0s 45us/step - loss: 0.4329 - acc: 0.8395
Epoch 84/1000
891/891 [==============================] - 0s 46us/step - loss: 0.4414 - acc: 0.8283
Epoch 85/1000
891/891 [==============================] - 0s 46us/step - loss: 0.4251 - acc: 0.8283
Epoch 86/1000
891/891 [==============================] - 0s 55us/step - loss: 0.4336 - acc: 0.8395
Epoch 87/1000
891/891 [==============================] - 0s 51us/step - loss: 0.4274 - acc: 0.8272
Epoch 88/1000
891/891 [==============================] - 0s 50us/step - loss: 0.4335 - acc: 0.8294
Epoch 89/1000
891/891 [==============================] - 0s 52us/step - loss: 0.4316 - acc: 0.8260
Epoch 90/1000
891/891 [==============================] - 0s 54us/step - loss: 0.4390 - acc: 0.8249
Epoch 91/1000
891/891 [==============================] - 0s 54us/step - loss: 0.4089 - acc: 0.8350
Epoch 92/1000
891/891 [==============================] - 0s 47us/step - loss: 0.4145 - acc: 0.8361
Epoch 93/1000
891/891 [=

891/891 [==============================] - 0s 52us/step - loss: 0.4262 - acc: 0.8283
Epoch 166/1000
891/891 [==============================] - 0s 48us/step - loss: 0.4266 - acc: 0.8204
Epoch 167/1000
891/891 [==============================] - 0s 49us/step - loss: 0.4131 - acc: 0.8249
Epoch 168/1000
891/891 [==============================] - 0s 49us/step - loss: 0.4243 - acc: 0.8328
Epoch 169/1000
891/891 [==============================] - 0s 47us/step - loss: 0.4221 - acc: 0.8249
Epoch 170/1000
891/891 [==============================] - 0s 45us/step - loss: 0.4148 - acc: 0.8305
Epoch 171/1000
891/891 [==============================] - 0s 44us/step - loss: 0.4068 - acc: 0.8384
Epoch 172/1000
891/891 [==============================] - 0s 45us/step - loss: 0.4072 - acc: 0.8305
Epoch 173/1000
891/891 [==============================] - 0s 44us/step - loss: 0.4189 - acc: 0.8238
Epoch 174/1000
891/891 [==============================] - 0s 50us/step - loss: 0.4141 - acc: 0.8260
Epoch 175/1000


891/891 [==============================] - 0s 50us/step - loss: 0.4005 - acc: 0.8373
Epoch 248/1000
891/891 [==============================] - 0s 47us/step - loss: 0.4121 - acc: 0.8272
Epoch 249/1000
891/891 [==============================] - 0s 46us/step - loss: 0.4200 - acc: 0.8294
Epoch 250/1000
891/891 [==============================] - 0s 50us/step - loss: 0.4080 - acc: 0.8316
Epoch 251/1000
891/891 [==============================] - 0s 53us/step - loss: 0.4244 - acc: 0.8249
Epoch 252/1000
891/891 [==============================] - 0s 51us/step - loss: 0.4084 - acc: 0.8350
Epoch 253/1000
891/891 [==============================] - 0s 50us/step - loss: 0.4085 - acc: 0.8272
Epoch 254/1000
891/891 [==============================] - 0s 46us/step - loss: 0.4172 - acc: 0.8294
Epoch 255/1000
891/891 [==============================] - 0s 51us/step - loss: 0.4101 - acc: 0.8361
Epoch 256/1000
891/891 [==============================] - 0s 48us/step - loss: 0.4146 - acc: 0.8215
Epoch 257/1000


...

891/891 [==============================] - 0s 49us/step - loss: 0.4006 - acc: 0.8429
Epoch 658/1000
891/891 [==============================] - 0s 45us/step - loss: 0.3998 - acc: 0.8350
Epoch 659/1000
891/891 [==============================] - 0s 45us/step - loss: 0.4199 - acc: 0.8328
Epoch 660/1000
891/891 [==============================] - 0s 45us/step - loss: 0.3977 - acc: 0.8406
Epoch 661/1000
891/891 [==============================] - 0s 43us/step - loss: 0.4011 - acc: 0.8328
Epoch 662/1000
891/891 [==============================] - 0s 44us/step - loss: 0.3941 - acc: 0.8406
Epoch 663/1000
891/891 [==============================] - 0s 44us/step - loss: 0.4091 - acc: 0.8440
Epoch 664/1000
891/891 [==============================] - 0s 45us/step - loss: 0.4015 - acc: 0.8294
Epoch 665/1000
891/891 [==============================] - 0s 45us/step - loss: 0.4051 - acc: 0.8384
Epoch 666/1000
891/891 [==============================] - 0s 45us/step - loss: 0.4018 - acc: 0.8305
Epoch 667/1000


891/891 [==============================] - 0s 46us/step - loss: 0.4137 - acc: 0.8305
Epoch 740/1000
891/891 [==============================] - 0s 43us/step - loss: 0.4028 - acc: 0.8418
Epoch 741/1000
891/891 [==============================] - 0s 42us/step - loss: 0.4075 - acc: 0.8395
Epoch 742/1000
891/891 [==============================] - 0s 42us/step - loss: 0.4026 - acc: 0.8272
Epoch 743/1000
891/891 [==============================] - 0s 44us/step - loss: 0.4038 - acc: 0.8395
Epoch 744/1000
891/891 [==============================] - 0s 47us/step - loss: 0.4011 - acc: 0.8373
Epoch 745/1000
891/891 [==============================] - 0s 52us/step - loss: 0.4115 - acc: 0.8395
Epoch 746/1000
891/891 [==============================] - 0s 57us/step - loss: 0.4016 - acc: 0.8406
Epoch 747/1000
891/891 [==============================] - 0s 64us/step - loss: 0.4058 - acc: 0.8451
Epoch 748/1000
891/891 [==============================] - 0s 58us/step - loss: 0.4046 - acc: 0.8451
Epoch 749/1000


891/891 [==============================] - 0s 55us/step - loss: 0.4160 - acc: 0.8272
Epoch 822/1000
891/891 [==============================] - 0s 50us/step - loss: 0.4100 - acc: 0.8238
Epoch 823/1000
891/891 [==============================] - 0s 49us/step - loss: 0.4158 - acc: 0.8339
Epoch 824/1000
891/891 [==============================] - 0s 49us/step - loss: 0.3959 - acc: 0.8373
Epoch 825/1000
891/891 [==============================] - 0s 51us/step - loss: 0.4051 - acc: 0.8316
Epoch 826/1000
891/891 [==============================] - 0s 50us/step - loss: 0.4150 - acc: 0.8260
Epoch 827/1000
891/891 [==============================] - 0s 48us/step - loss: 0.4152 - acc: 0.8305
Epoch 828/1000
891/891 [==============================] - 0s 52us/step - loss: 0.4084 - acc: 0.8361
Epoch 829/1000
891/891 [==============================] - 0s 49us/step - loss: 0.4167 - acc: 0.8373
Epoch 830/1000
891/891 [==============================] - 0s 54us/step - loss: 0.4022 - acc: 0.8339
Epoch 831/1000


891/891 [==============================] - 0s 50us/step - loss: 0.3923 - acc: 0.8361
Epoch 904/1000
891/891 [==============================] - 0s 48us/step - loss: 0.4008 - acc: 0.8507
Epoch 905/1000
891/891 [==============================] - 0s 51us/step - loss: 0.4187 - acc: 0.8260
Epoch 906/1000
891/891 [==============================] - 0s 45us/step - loss: 0.3968 - acc: 0.8238
Epoch 907/1000
891/891 [==============================] - 0s 52us/step - loss: 0.4004 - acc: 0.8462
Epoch 908/1000
891/891 [==============================] - 0s 56us/step - loss: 0.4165 - acc: 0.8294
Epoch 909/1000
891/891 [==============================] - 0s 55us/step - loss: 0.4081 - acc: 0.8328
Epoch 910/1000
891/891 [==============================] - 0s 46us/step - loss: 0.4010 - acc: 0.8451
Epoch 911/1000
891/891 [==============================] - 0s 48us/step - loss: 0.4072 - acc: 0.8316
Epoch 912/1000
891/891 [==============================] - 0s 46us/step - loss: 0.4207 - acc: 0.8316
Epoch 913/1000


891/891 [==============================] - 0s 47us/step - loss: 0.4070 - acc: 0.8361
Epoch 986/1000
891/891 [==============================] - 0s 46us/step - loss: 0.3981 - acc: 0.8474
Epoch 987/1000
891/891 [==============================] - 0s 43us/step - loss: 0.3976 - acc: 0.8440
Epoch 988/1000
891/891 [==============================] - 0s 42us/step - loss: 0.3993 - acc: 0.8384
Epoch 989/1000
891/891 [==============================] - 0s 44us/step - loss: 0.4081 - acc: 0.8283
Epoch 990/1000
891/891 [==============================] - 0s 47us/step - loss: 0.3921 - acc: 0.8395
Epoch 991/1000
891/891 [==============================] - 0s 51us/step - loss: 0.4128 - acc: 0.8328
Epoch 992/1000
891/891 [==============================] - 0s 48us/step - loss: 0.4107 - acc: 0.8316
Epoch 993/1000
891/891 [==============================] - 0s 42us/step - loss: 0.3973 - acc: 0.8294
Epoch 994/1000
891/891 [==============================] - 0s 41us/step - loss: 0.4195 - acc: 0.8339
Epoch 995/1000


In [25]:
# Predicting the Test set results, using the 0.5 threshold and get true or false values to build a confusion matrix.
y_pred = classifier.predict(X_test)
y_pred = pd.DataFrame(y_pred)


In [26]:
### Converting the prediction to a 0/1 value at a 0.5 threshold.
def binary(x):
    if x > 0.5:
        return 1
    else:
        return 0

y_pred = y_pred[0].apply(binary)


In [27]:
 #Since i deleted the dataset above, i load it again to get the PassengerId.

Ids = pd.read_csv('data/test.csv')


In [28]:
submission = pd.DataFrame({
        "PassengerId": Ids["PassengerId"],
        "Survived": y_pred
    })
    
submission.to_csv('grimreaper94_keras_submit.csv', index=False)